In [1]:
!pip install mosek


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import torch.nn.functional as F
import networkx as nx
import numpy as np
import torch.optim as optim
from itertools import combinations
import pickle
import cvxpy as cp

In [3]:
# Define a graph
# n = 50  # number of nodes
# m = 613  # number of edges
# G = nx.gnm_random_graph(n, m, seed = 0)

#G = nx.gnm_random_graph(1200, 4000, seed = 0)

#G = nx.gnm_random_graph(n, m, seed = 4) # This graph has a MIS of 5, and it is hard to find!!! They are {0, 2, 9, 10, 11} and {0, 2, 8, 9, 11}
# Create a G(n, m) random graph
def destringizer(value):
    try:
        # Try to convert the string to an integer
        return int(value)
    except ValueError:
        try:
            # Try to convert the string to a float
            return float(value)
        except ValueError:
            # If conversion fails, return the original string
            return value

# Load the .gml graph using nx.read_gml and provide the destringizer function
#G = nx.read_gml("GNM_n50_m613_seed9.gml", destringizer=destringizer)
#G = nx.read_gml("/content/GNM_n500_m62375_seed2.gml", destringizer=destringizer)

#G = nx.read_gpickle("/content/CBS_k3_n100_m403_b10_18.gpickle")

file_path = "./graphs/satlib/CBS_k3_n100_m449_b90_84.gpickle"

with open(file_path, 'rb') as f:
    G = pickle.load(f)

n = len(G.nodes)
m = len(G.edges)
#print(G.edges)
complement_G = nx.complement(G)

print(n,m)



### Visualize the graph
#nx.draw(G, with_labels=True, font_weight='bold')

1347 5739


In [4]:
import numpy as np
import networkx as nx
from mosek.fusion import Model, Domain, ObjectiveSense, Matrix, Expr

# Get the adjacency matrix
A = np.array(nx.adjacency_matrix(G).todense())

# Get the number of nodes
n = len(G.nodes)

matrix_of_ones = np.ones((n, n))

# Create a MOSEK model
with Model("SDP_init") as m:

    # Create the matrix variable X
    X = m.variable('X', [n, n], Domain.inPSDCone())

    #ones_mat = mosek.fusion.Expression.constant(1.0, n, n)
    objective_expr = Expr.dot(matrix_of_ones, X)

    # Set the objective: maximize the trace of X
    m.objective('obj', ObjectiveSense.Maximize, objective_expr)

    # Add constraints: trace(X) == 1 and X_{i,j} == 0 for all {i, j} in E
    m.constraint('tr', Expr.sum(X.diag()), Domain.equalsTo(1.0))
    for edge in G.edges:
        i, j = edge
        m.constraint(f'con_{i}_{j}', X.index(i, j), Domain.equalsTo(0.0))
        m.constraint(f'con_{j}_{i}', X.index(j, i), Domain.equalsTo(0.0))

    # Solve the problem
    m.solve()

    # # Get the solution
    # solution = m.getSolution()
    # if solution is not None:
    #     # Get the objective value from the solution object
    #     optimal_value = solution.getPrimalObjective()
    #     print("Optimal value:", optimal_value)
    # else:
    #     print("No solution found.")

    # Get the solution
    X_solution = X.level()
    #X_solution = m.getPrimalSolution()

# Display the results
#print("Optimal value:", m.primalObjValue())
print("Optimal solution (X):")
print(X_solution.shape)


Optimal solution (X):
(1814409,)


In [5]:
#### reshape X:
X_solution_reshaped = X_solution.reshape(n, n)
# for node in G.nodes:
#   print(node, X_solution_reshaped[node,node])

# Create a matrix of ones with the same shape as X_solution_reshaped
ones_matrix = np.ones((n, n))

# Compute the element-wise multiplication
ones_X_product = ones_matrix @ X_solution_reshaped

# Compute the trace by summing the diagonal elements
trace_ones_X_product = np.sum(np.diag(ones_X_product))

print("Trace of (ones * X_solution_reshaped):", trace_ones_X_product)


Trace of (ones * X_solution_reshaped): 449.00000005222614


In [6]:
# ####################### coding up the SDP for initialization and encoding...
# # Get the adjacency matrix
# A = np.array(nx.adjacency_matrix(G).todense())

# # Define the SDP optimization variable
# X_SDP = cp.Variable((n, n), symmetric=True)

# # Formulate the optimization problem
# objective = cp.Maximize(cp.trace(np.ones((n, n)) @ X_SDP))
# constraints = [
#     cp.trace(X_SDP) == 1,
#     X_SDP >> 0,
# ]

# # Add constraints for X_{i,j} = 0 for all {i, j} in E
# for edge in G.edges:
#     i, j = edge
#     constraints.append(X_SDP[i, j] == 0)
#     constraints.append(X_SDP[j, i] == 0)

# # Create the CVXPY problem
# problem = cp.Problem(objective, constraints)

# # Solve the problem
# problem.solve()

# # Extract the solution
# X_SDP_solution = X_SDP.value

# # Display the results
# print("Optimal value:", problem.value)

In [7]:
######### Coding up W_SDP_uv for (u,v) not in E, vector s_SDP_v (for the first term from the diagonal of X_SDP), and X_SDP_init:

## Coding W_SDP_uv:
W_SDP_uv = torch.zeros(n,n)

# create a dictionary of key = edge (u,v) in G' and value is equation to X(u,v)
G_hat_edges_strength_dict = {}

for v in range(n):
  for u in range(n):
    if (v,u) not in G.edges() and u != v:
      G_hat_edges_strength_dict[(v,u)] =  X_solution_reshaped[v,u]

# Normalize
values_array = np.array(list(G_hat_edges_strength_dict.values()))
normalized_values_MINMAX = (values_array - np.min(values_array)) / (np.max(values_array) - np.min(values_array))
G_hat_edges_strength_dict_normalized_MINMAX = {key: normalized_values_MINMAX[i] for i, key in enumerate(G_hat_edges_strength_dict)} ######## USE THIS

# Assign to W_SDP_uv
for v in range(n):
  for u in range(n):
    if (v,u) not in G.edges() and v != u:
      W_SDP_uv[v,u] = G_hat_edges_strength_dict_normalized_MINMAX[(v,u)]

print(W_SDP_uv[1,88])
print(G_hat_edges_strength_dict_normalized_MINMAX[(1,88)])
print(X_solution_reshaped[1,88])

## Coding s_SDP_v and X_SDP_init:

sol_diag = np.diag(X_solution_reshaped)
s_SDP_v = sol_diag / np.max(sol_diag)
X_SDP_init = (sol_diag - np.min(sol_diag)) / (np.max(sol_diag) - np.min(sol_diag))
print(s_SDP_v[0:10])
print(X_SDP_init[0:10])

### Converting to torch tensors: Convert s_SDP_v and W_SDP_uv to torch tensors
s_SDP_v_tensor = torch.tensor(s_SDP_v, dtype=torch.float32)
X_SDP_init_tensor = torch.tensor(X_SDP_init, dtype=torch.float32)
W_SDP_uv_tensor = torch.tensor(W_SDP_uv, dtype=torch.float32)

tensor(0.3860)
0.3859703183523298
0.0003600230434673559
[0.45784384 0.67005924 0.52651996 0.25576945 0.64484805 0.75380548
 0.49403743 0.70070578 0.45967978 0.61322618]
[0.35288514 0.60618437 0.43485661 0.11169017 0.57609243 0.70614347
 0.39608563 0.64276393 0.35507651 0.53834872]


/tmp/ipykernel_61945/4058834096.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  W_SDP_uv_tensor = torch.tensor(W_SDP_uv, dtype=torch.float32)


In [8]:
# ### Obtain the A_G matrix

# adjacency_matrix = nx.adjacency_matrix(G)
# adjacency_matrix_dense = adjacency_matrix.todense()
# adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

# ### Obtain the A_G_hat matrix

# adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
# adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
# adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)

# def loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma, beta):
#     ## without edges of the comp graph:
#     #loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X)

#     ## with edges of the comp graph:
#     loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)
#     return loss

# # # test function:
# # Matrix_X = torch.rand((len(G.nodes),1), requires_grad=True)
# # print("testing the function: ", loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma=5, beta = 1))


In [9]:
########## DEFINING THE SDP ENCODED LOSS:

### Obtain the A_G matrix

adjacency_matrix = nx.adjacency_matrix(G)
adjacency_matrix_dense = adjacency_matrix.todense()
adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

### Obtain the A_G_hat matrix

adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)



def loss_function_with_SDP(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, s_SDP_v_tensor, W_SDP_uv_tensor, gamma, beta):
    ## without edges of the comp graph:
    #loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X)
    beta = 1
    ## with edges of the comp graph:
    #loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)
    loss = -(Matrix_X.T @ s_SDP_v_tensor) + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp*W_SDP_uv_tensor) @ Matrix_X)
    return loss

# # test function:
# Matrix_X = torch.rand((len(G.nodes),1), requires_grad=True)
# print("testing the function: ", loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma=5, beta = 1))


In [10]:
# # Constructing the constant Hessian and looking at the eigen values: This depends on the connectivity of graph... The more edges, the worst, which we already know
# gamma = 90 # I select this from the best solution from ReduMIS: Double check the bounds with Rongrong
# beta = 1
# Hessian = torch.zeros(n, n)
# complement_G = nx.complement(G)

# Hessian_with_third = (gamma/2)*adjacency_matrix_tensor - (beta/2)*adjacency_matrix_tensor_comp
# Hessain_without    = (gamma/2)*adjacency_matrix_tensor

# # Compute eigenvalues
# eigenvalues_with, _ = torch.linalg.eig(Hessian_with_third)

# # Extract the real part of the eigenvalues
# real_eigenvalues_with = eigenvalues_with.real


# # Compute eigenvalues
# eigenvalues_without, _ = torch.linalg.eig(Hessain_without)

# # Extract the real part of the eigenvalues
# real_eigenvalues_without = eigenvalues_without.real

# #print("Eigenvalues without:", real_eigenvalues_without)

# # Count positive values
# positive_count_with = torch.sum(real_eigenvalues_with > 0).item()

# # Count negative values
# negative_count_with = torch.sum(real_eigenvalues_with < 0).item()

# print("Number of positive values with:", positive_count_with)
# print("Number of negative values with:", negative_count_with)

# # Count positive values
# positive_count_without = torch.sum(real_eigenvalues_without > 0).item()

# # Count negative values
# negative_count_without = torch.sum(real_eigenvalues_without < 0).item()

# print("Number of positive eigen values without:", positive_count_without)
# print("Number of negative eigen values without:", negative_count_without)

# print(real_eigenvalues_without)


In [11]:
# This is a function to double check if the MIS we find is a valid IS and/or MIS

def MIS_checker(MIS_list, G):
  pairs = list(combinations(MIS_list, 2))
  IS_CHECKER = True
  if len(MIS_list)>1:
    for pair in pairs:
      if (pair[0], pair[1]) in G.edges or (pair[1], pair[0]) in G.edges:
        IS_CHECKER =  False
        break
  # is it a MIS or IS?
  # if MIS_list is an IS:
  if IS_CHECKER:
    #print("Its an IS")
    # obtain a list of all niebouring nodes
    list_of_all_Ns_in_MIS_list = []
    for node in MIS_list:
      list_of_all_Ns_in_MIS_list.append(list(G.neighbors(node)))

    flattened_list_of_all_Ns_in_MIS_list = [item for sublist in list_of_all_Ns_in_MIS_list for item in sublist]
    #print(flattened_list_of_all_Ns_in_MIS_list)

    list_of_nodes_inG_but_not_in_MIStoCHeck = [item for item in list(G.nodes) if item not in MIS_list]

    MIS_CHECKER = True
    for node in list_of_nodes_inG_but_not_in_MIStoCHeck:
      if node not in flattened_list_of_all_Ns_in_MIS_list:
        MIS_CHECKER = False
        break
  else:
    IS_CHECKER = False
    MIS_CHECKER = False
  return IS_CHECKER, MIS_CHECKER

# test function: use G = nx.gnm_random_graph(15, 52, seed = 4)
print(MIS_checker([0, 1, 2, 3, 4], G))  # Not a valid IS

print(MIS_checker([1, 9], G))  # A valid IS

print(MIS_checker([0, 2, 9, 10, 11], G))  # A valid MIS

print(MIS_checker([0, 2, 9, 10], G))  # A valid IS





(False, False)
(True, False)
(False, False)
(False, False)


In [12]:
start_value = 800
end_value = 100000
step = 800

Indicies_for_complete_restart = list(range(start_value, end_value + 1, step))

print(Indicies_for_complete_restart)

[800, 1600, 2400, 3200, 4000, 4800, 5600, 6400, 7200, 8000, 8800, 9600, 10400, 11200, 12000, 12800, 13600, 14400, 15200, 16000, 16800, 17600, 18400, 19200, 20000, 20800, 21600, 22400, 23200, 24000, 24800, 25600, 26400, 27200, 28000, 28800, 29600, 30400, 31200, 32000, 32800, 33600, 34400, 35200, 36000, 36800, 37600, 38400, 39200, 40000, 40800, 41600, 42400, 43200, 44000, 44800, 45600, 46400, 47200, 48000, 48800, 49600, 50400, 51200, 52000, 52800, 53600, 54400, 55200, 56000, 56800, 57600, 58400, 59200, 60000, 60800, 61600, 62400, 63200, 64000, 64800, 65600, 66400, 67200, 68000, 68800, 69600, 70400, 71200, 72000, 72800, 73600, 74400, 75200, 76000, 76800, 77600, 78400, 79200, 80000, 80800, 81600, 82400, 83200, 84000, 84800, 85600, 86400, 87200, 88000, 88800, 89600, 90400, 91200, 92000, 92800, 93600, 94400, 95200, 96000, 96800, 97600, 98400, 99200, 100000]


In [13]:
# Optimization loop:
# Initialization:
torch.manual_seed(115)
#Matrix_X = torch.zeros((n), requires_grad=True)
#Matrix_X = torch.ones((n), requires_grad=True)
#Matrix_X = torch.rand((n), requires_grad=True)

lower_bound = 0.01
upper_bound = 0.2
# Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
#Matrix_X = 1.0*torch.ones(n)
Matrix_X = X_SDP_init_tensor
Matrix_X.requires_grad_(True)


# dict for saving... I am thinking of diversification by looking at previous initializations... Still under investigation
#dict_of_MISs = {}

# list of MISs to apply the 2-opt alg
list_of_MISs_to_apply_2opt = []

# This is obtained to get a sense of how far are we from the init
X_ini  = Matrix_X.data.clone()
#gamma = torch.tensor(50.0, requires_grad=True)

learning_rate_alpha = 0.5
number_of_iterations_T = 10000


# Define Optimizer over matrix X
#optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha, weight_decay=10)
optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)
#optimizer = optim.SGD([Matrix_X], lr=learning_rate_alpha)

Best_MIS = 0
for iteration_t in range(number_of_iterations_T):

    loss = loss_function_with_SDP(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X,s_SDP_v_tensor, W_SDP_uv_tensor, gamma = 250, beta = 1)

    optimizer.zero_grad()  # Clear gradients for the next iteration
    loss.backward()  # Backpropagation
    optimizer.step()  # Update the parameters

    # Box-constraining:
    Matrix_X.data[Matrix_X>=1] =1
    Matrix_X.data[Matrix_X<=0] =0


    # Report the current MIS
    MIS = []
    for node in G.nodes:
        if Matrix_X[node] >0.1:
          MIS.append(node)

    # If no MIS, move one
    if MIS_checker(MIS, G)[0] is False: MIS = []

    ## Measuring the gradient sparsity: The results is that the gradient is never sparse
    #sparsity_ratio = ( Matrix_X.grad == 0.0).sum().item() / n
    #print(sparsity_ratio)

    # Iteration logger every XX iterations:
    if (iteration_t + 1) % 200 == 0:
        print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}")

        #print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}, grad norm: {torch.norm(Matrix_X.grad).item()}")
        #print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}")
    if MIS_checker(MIS, G)[1] is True:
      list_of_MISs_to_apply_2opt.append(MIS)

      if len(MIS) > Best_MIS: # This is to save our current best so far
        Best_MIS = len(MIS)

      l2_norm = torch.norm(Matrix_X - X_ini, p=2) # If a solution is found, measure its distance from initialization:
      print("+++++++++++ A MIS is found with size: ", [[[len(MIS)]]], "++++ BEST so far",[Best_MIS], "Dist from Init: =",l2_norm.item() , "++++++++++++", [MIS])


    #   #### Local restart every XX iterations:

    #   Matrix_X.data[Matrix_X>0] = 0.8*torch.rand((1))
    #   Matrix_X.data[Matrix_X==0] = 0.2*torch.rand((1))
    #   X_ini = Matrix_X.data.clone()
    #   optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)

    # if iteration_t in Indicies_for_complete_restart:
    #     print("****************&&&&&&&&&&&&&****************&&&&&&&&&&&&& Restarting X Completely at RANDOM &&&&&&&&&&&&&&***************************&&&&&&&&&&&&&")
    #     Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
    #     Matrix_X.requires_grad_(True)
    #     X_ini = Matrix_X.data.clone()
    #     optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)


      # # Restart X and the optimizer to search at a different point in [0,1]^n

      # #lower_bound = 0.2
      # #upper_bound = 1.0
      # Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
      # Matrix_X.requires_grad_(True)


      # #Matrix_X = torch.rand((n), requires_grad=True)
      # X_ini = Matrix_X.data.clone()
      # dict_of_inits[X_ini] = MIS
      # optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)
      # #break

      ##### Restart around the same point that we get from the SDP:
      mean_vector = X_SDP_init_tensor
      covariance_matrix = 1*torch.eye(len(mean_vector))
      Matrix_X = torch.normal(mean=mean_vector, std=torch.sqrt(torch.diag(covariance_matrix)))
      Matrix_X = Matrix_X.detach().clone()
      #Matrix_X.data[Matrix_X>=1] =1
      #Matrix_X.data[Matrix_X<=0] =0
      Matrix_X.requires_grad_(True)
      optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)
      #optimizer = optim.SGD([Matrix_X], lr=learning_rate_alpha)



# # Visualize the solution:
# node_colors = ['green' if node in MIS else 'blue' for node in G.nodes]
# nx.draw(G, with_labels=True, font_weight='bold', node_color=node_colors)

/tmp/ipykernel_61945/3816875553.py:23: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  loss = -(Matrix_X.T @ s_SDP_v_tensor) + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp*W_SDP_uv_tensor) @ Matrix_X)


+++++++++++ A MIS is found with size:  [[[440]]] ++++ BEST so far [440] Dist from Init: = 14.244668006896973 ++++++++++++ [[1, 5, 7, 9, 13, 16, 18, 22, 25, 27, 31, 33, 36, 41, 43, 46, 49, 53, 54, 57, 62, 63, 71, 72, 76, 80, 83, 86, 88, 91, 94, 97, 100, 104, 106, 109, 111, 114, 118, 121, 124, 127, 130, 133, 136, 140, 142, 146, 148, 152, 153, 157, 159, 163, 165, 170, 172, 176, 177, 181, 185, 187, 189, 193, 195, 198, 201, 205, 207, 210, 213, 216, 219, 223, 227, 229, 232, 234, 239, 240, 247, 249, 253, 255, 259, 262, 266, 268, 271, 275, 276, 280, 282, 286, 290, 291, 295, 297, 300, 303, 307, 311, 314, 320, 321, 326, 327, 330, 333, 336, 340, 343, 346, 349, 351, 354, 357, 360, 364, 366, 369, 374, 377, 380, 382, 385, 389, 392, 393, 398, 401, 402, 407, 408, 411, 415, 419, 420, 425, 427, 429, 434, 436, 439, 442, 446, 448, 451, 454, 456, 459, 463, 466, 468, 472, 475, 479, 481, 484, 488, 489, 493, 496, 500, 502, 505, 509, 511, 515, 517, 521, 523, 525, 529, 532, 534, 537, 543, 548, 550, 552, 556, 55

In [14]:
print(list_of_MISs_to_apply_2opt[2])
print(len(list_of_MISs_to_apply_2opt))

# unique_lists = list(set(tuple(lst) for lst in list_of_MISs_to_apply_2opt))

# unique_lists = [list(lst) for lst in unique_lists]

# print(len(unique_lists))

[1, 5, 8, 10, 12, 15, 18, 22, 25, 27, 31, 33, 36, 41, 43, 46, 49, 51, 54, 57, 62, 63, 68, 71, 72, 76, 83, 86, 88, 91, 94, 98, 100, 102, 106, 110, 111, 114, 118, 121, 124, 128, 129, 133, 137, 140, 142, 146, 148, 152, 153, 159, 163, 166, 170, 172, 176, 177, 181, 183, 188, 189, 193, 196, 198, 201, 205, 207, 210, 213, 216, 219, 223, 226, 229, 233, 236, 238, 240, 243, 247, 249, 253, 255, 259, 263, 264, 269, 271, 273, 278, 280, 282, 286, 290, 291, 295, 297, 300, 303, 307, 311, 314, 320, 321, 324, 327, 332, 333, 336, 340, 343, 347, 349, 351, 356, 357, 360, 364, 366, 370, 374, 377, 380, 382, 385, 387, 392, 393, 398, 400, 404, 407, 408, 413, 415, 417, 420, 425, 427, 429, 432, 436, 439, 442, 446, 448, 452, 455, 456, 459, 463, 466, 468, 471, 475, 478, 481, 484, 487, 489, 493, 496, 500, 502, 505, 509, 510, 515, 517, 519, 523, 525, 529, 532, 535, 538, 543, 548, 550, 554, 557, 558, 562, 566, 567, 572, 574, 577, 579, 585, 588, 592, 595, 599, 602, 603, 607, 610, 612, 617, 619, 622, 625, 628, 631, 635,

In [15]:
# coding up the 2-opt algorithm:

import itertools


def solution_improve_alg_MIS(MIS_list, G):
    """

    :param MIS: the MIS solution list
    :param G: graph
    :return: increased size of the MIS if exists
    """

    def intersection(lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3

    ### inputs: Graph, MIS
    #MIS = list(np.argwhere(X_star_thresholded == 1).reshape(length, ))
    MIS = MIS_list
    length = len(MIS)
    A = nx.to_numpy_array(G)
    list_of_replacement = []
    improv2_cntr = 0
    for MIS_node in MIS:
        # get combinations of nieghbors
        Neighbors = list(G[MIS_node])
        paris_of_Nieghbours = list(itertools.combinations(Neighbors, 2))
        for pair in paris_of_Nieghbours:
            # if they are not nighbours
            if A[pair[0], pair[1]] == 0:  # and len(intersection(MIS,[pair[0]])) == 0 and len(intersection(MIS,[pair[1]])) == 0:
                # get nighbors of node_j and node_l
                Neighbors_pair_0 = list(G[pair[0]])
                Neighbors_pair_1 = list(G[pair[1]])
                # tight-1 check on node_j and node_l
                if len(intersection(MIS, Neighbors_pair_0)) == 1 and len(intersection(MIS, Neighbors_pair_1)) == 1:
                    #improv2_cntr = improv2_cntr + 1
                    # remove node
                    list_of_replacement.append([MIS_node, pair[0], pair[1]])
                    #print("A replacement is taking place... node {} is removed and nodes {} and {} are added".format(
                    #    MIS_node, pair[0], pair[1]))
                    break
    ### here we need to loop over the potential list of replacements, remove and add, then check if we still have a MIS to update accordingly

    for potential_impr in list_of_replacement:
        # remove
        MIS_temp = MIS
        MIS_temp.remove(potential_impr[0])
        MIS_temp.append(potential_impr[1])
        MIS_temp.append(potential_impr[2])
        if MIS_checker(MIS_temp,G) is True:
            improv2_cntr = improv2_cntr + 1
            MIS = MIS_temp
            print("Solution Improvement Replacement is taking place without harm ... node {} is removed and nodes {} and {} are added".format(
                potential_impr[0], potential_impr[1], potential_impr[2]))
        if MIS_checker(MIS_temp,G) is False:
            # reversing the changes in case of problems
            MIS_temp.append(potential_impr[0])
            MIS_temp.remove(potential_impr[1])
            MIS_temp.remove(potential_impr[2])
            MIS = MIS_temp
            print("The potential nodes replacments are no good. Hence, making no changes")


    return MIS

In [17]:
from operator import length_hint

max_val = 0
for MIS_instant in list_of_MISs_to_apply_2opt:

    length_before = len(MIS_instant)
    length_after = len(solution_improve_alg_MIS(MIS_instant, G))

    max_val = max(length_after, max_val)
    print("BEFORE: ", length_before, "AFTER: ", length_after, "improvements = ", length_after-length_before)

    #print("BEFORE: ", len(MIS_instant), "AFTER: ", len(solution_improve_alg_MIS(MIS_instant, G)), "improvements = ", len(solution_improve_alg_MIS(MIS, G))-len(MIS))
    #print("BEFORE: ", len(MIS_instant), "AFTER: ", len(solution_improve_alg_MIS(MIS_instant, G)))



print(max_val)



BEFORE:  440 AFTER:  440 improvements =  0
BEFORE:  438 AFTER:  440 improvements =  2
BEFORE:  438 AFTER:  438 improvements =  0
BEFORE:  442 AFTER:  442 improvements =  0
BEFORE:  438 AFTER:  438 improvements =  0
BEFORE:  439 AFTER:  439 improvements =  0
BEFORE:  440 AFTER:  442 improvements =  2
BEFORE:  434 AFTER:  434 improvements =  0
BEFORE:  438 AFTER:  438 improvements =  0
BEFORE:  438 AFTER:  438 improvements =  0
BEFORE:  439 AFTER:  439 improvements =  0
BEFORE:  439 AFTER:  439 improvements =  0
BEFORE:  435 AFTER:  435 improvements =  0
BEFORE:  436 AFTER:  436 improvements =  0
BEFORE:  437 AFTER:  437 improvements =  0
BEFORE:  439 AFTER:  440 improvements =  1
BEFORE:  438 AFTER:  438 improvements =  0
BEFORE:  441 AFTER:  441 improvements =  0
BEFORE:  438 AFTER:  438 improvements =  0
BEFORE:  438 AFTER:  439 improvements =  1
BEFORE:  438 AFTER:  438 improvements =  0
BEFORE:  437 AFTER:  437 improvements =  0
BEFORE:  436 AFTER:  436 improvements =  0
BEFORE:  43